언니의 연습장에 습격!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [ ]:
data_jeju = pd.read_csv('/content/drive/My Drive/SAMDASOODA_DACON/data/card_data/data_jeju.csv', parse_dates=['REG_YYMM'], index_col= 0)

In [ ]:
data_jeju

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
1454855,201901,제주,서귀포시,건강보조식품 소매업,부산,사하구,50s,2,4,4,1900000,4
1454856,201901,제주,서귀포시,건강보조식품 소매업,인천,서구,50s,2,4,3,1300000,2
1454857,201901,제주,서귀포시,건강보조식품 소매업,제주,서귀포시,30s,1,2,3,438000,3
1454858,201901,제주,서귀포시,건강보조식품 소매업,제주,서귀포시,30s,2,2,4,552200,3
1454859,201901,제주,서귀포시,건강보조식품 소매업,제주,서귀포시,40s,1,2,3,908000,3
...,...,...,...,...,...,...,...,...,...,...,...,...
24597421,202003,제주,제주시,휴양콘도 운영업,서울,노원구,50s,1,4,3,79500,3
24597422,202003,제주,제주시,휴양콘도 운영업,제주,제주시,30s,1,2,4,1656000,5
24597423,202003,제주,제주시,휴양콘도 운영업,제주,제주시,40s,1,3,3,304560,3
24597424,202003,제주,제주시,휴양콘도 운영업,제주,제주시,50s,2,4,4,60000,4


In [ ]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [ ]:
data_jeju = data_jeju.fillna('')
data_jeju['year'] = data_jeju['REG_YYMM'].apply(lambda x: grap_year(x))
data_jeju['month'] = data_jeju['REG_YYMM'].apply(lambda x: grap_month(x))
data_jeju = data_jeju.drop(['REG_YYMM'], axis=1)

In [ ]:
df = data_jeju.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [ ]:
# dtypes = df.dtypes

# encoders = {}
# for column in df.columns:
#     if str(dtypes[column]) == 'object':
#         encoder = LabelEncoder()
#         encoder.fit(df[column])
#         encoders[column] = encoder
        
# df_num = df.copy()        
# for column in encoders.keys():
#     encoder = encoders[column]
#     df_num[column] = encoder.transform(df[column])

In [ ]:
df.columns

Index(['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD',
       'FLC', 'year', 'month', 'CSTMR_CNT', 'AMT', 'CNT'],
      dtype='object')

In [ ]:
df = df.join(pd.get_dummies(df.STD_CLSS_NM, prefix='업종'))
df = df.join(pd.get_dummies(df.AGE, prefix='나이'))
df = df.join(pd.get_dummies(df.SEX_CTGO_CD, prefix='성별'))
df = df.join(pd.get_dummies(df.FLC, prefix='가구'))

In [ ]:
cols = ['STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD','FLC']
df.drop(cols, axis=1, inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79154 entries, 0 to 79153
Data columns (total 61 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   CARD_SIDO_NM              79154 non-null  object
 1   HOM_SIDO_NM               79154 non-null  object
 2   year                      79154 non-null  int64 
 3   month                     79154 non-null  int64 
 4   CSTMR_CNT                 79154 non-null  int64 
 5   AMT                       79154 non-null  int64 
 6   CNT                       79154 non-null  int64 
 7   업종_건강보조식품 소매업             79154 non-null  uint8 
 8   업종_골프장 운영업                79154 non-null  uint8 
 9   업종_과실 및 채소 소매업            79154 non-null  uint8 
 10  업종_관광 민예품 및 선물용품 소매업      79154 non-null  uint8 
 11  업종_그외 기타 분류안된 오락관련 서비스업   79154 non-null  uint8 
 12  업종_그외 기타 스포츠시설 운영업        79154 non-null  uint8 
 13  업종_그외 기타 종합 소매업           79154 non-null  uint8 
 14  업종_기타 대형 종합 소매업       

In [ ]:
# df_distance = df.copy()
df['SIDOCHAI'] = 0

a = df['CARD_SIDO_NM'] != df['HOM_SIDO_NM']
df.loc[a, 'SIDOCHAI'] = 1

In [ ]:
df

,CARD_SIDO_NM,HOM_SIDO_NM,year,month,CSTMR_CNT,AMT,CNT,업종_건강보조식품 소매업,업종_골프장 운영업,업종_과실 및 채소 소매업,업종_관광 민예품 및 선물용품 소매업,업종_그외 기타 분류안된 오락관련 서비스업,업종_그외 기타 스포츠시설 운영업,업종_그외 기타 종합 소매업,업종_기타 대형 종합 소매업,업종_기타 수상오락 서비스업,업종_기타 외국식 음식점업,업종_기타 주점업,업종_기타음식료품위주종합소매업,업종_내항 여객 운송업,업종_마사지업,업종_면세점,업종_비알콜 음료점업,업종_빵 및 과자류 소매업,업종_서양식 음식점업,업종_수산물 소매업,업종_슈퍼마켓,업종_스포츠 및 레크레이션 용품 임대업,업종_여관업,업종_여행사업,업종_욕탕업,업종_육류 소매업,업종_일반유흥 주점업,업종_일식 음식점업,업종_자동차 임대업,업종_전시 및 행사 대행업,업종_정기 항공 운송업,업종_중식 음식점업,업종_차량용 가스 충전업,업종_차량용 주유소 운영업,업종_체인화 편의점,업종_택시 운송업,업종_피자 햄버거 샌드위치 및 유사 음식점업,업종_한식 음식점업,업종_호텔업,업종_화장품 및 방향제 소매업,업종_휴양콘도 운영업,나이_10s,나이_20s,나이_30s,나이_40s,나이_50s,나이_60s,나이_70s,성별_1,성별_2,가구_1,가구_2,가구_3,가구_4,가구_5,SIDOCHAI
0,제주,강원,2019,5,3,700000,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1
1,제주,경기,2019,3,3,2000000,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1
2,제주,경기,2019,10,3,1900000,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1
3,제주,경기,2019,4,4,3100000,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1
4,제주,경기,2019,6,4,2600000,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79149,제주,충북,2019,3,3,557200,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1
79150,제주,충북,2019,4,3,430200,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1
79151,제주,충북,2019,5,6,922280,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1
79152,제주,충북,2019,11,6,829520,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1


In [ ]:
df['PERSON'] = 0
b = df['CSTMR_CNT']>df['CNT']
c = df['CSTMR_CNT']<=df['CNT']
df.loc[b,'PERSON'] = df['CNT']
df.loc[c,'PERSON'] = df['CSTMR_CNT']

In [ ]:
df

,CARD_SIDO_NM,HOM_SIDO_NM,year,month,CSTMR_CNT,AMT,CNT,업종_건강보조식품 소매업,업종_골프장 운영업,업종_과실 및 채소 소매업,업종_관광 민예품 및 선물용품 소매업,업종_그외 기타 분류안된 오락관련 서비스업,업종_그외 기타 스포츠시설 운영업,업종_그외 기타 종합 소매업,업종_기타 대형 종합 소매업,업종_기타 수상오락 서비스업,업종_기타 외국식 음식점업,업종_기타 주점업,업종_기타음식료품위주종합소매업,업종_내항 여객 운송업,업종_마사지업,업종_면세점,업종_비알콜 음료점업,업종_빵 및 과자류 소매업,업종_서양식 음식점업,업종_수산물 소매업,업종_슈퍼마켓,업종_스포츠 및 레크레이션 용품 임대업,업종_여관업,업종_여행사업,업종_욕탕업,업종_육류 소매업,업종_일반유흥 주점업,업종_일식 음식점업,업종_자동차 임대업,업종_전시 및 행사 대행업,업종_정기 항공 운송업,업종_중식 음식점업,업종_차량용 가스 충전업,업종_차량용 주유소 운영업,업종_체인화 편의점,업종_택시 운송업,업종_피자 햄버거 샌드위치 및 유사 음식점업,업종_한식 음식점업,업종_호텔업,업종_화장품 및 방향제 소매업,업종_휴양콘도 운영업,나이_10s,나이_20s,나이_30s,나이_40s,나이_50s,나이_60s,나이_70s,성별_1,성별_2,가구_1,가구_2,가구_3,가구_4,가구_5,SIDOCHAI,PERSON
0,제주,강원,2019,5,3,700000,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,3
1,제주,경기,2019,3,3,2000000,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,3
2,제주,경기,2019,10,3,1900000,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,3
3,제주,경기,2019,4,4,3100000,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,4
4,제주,경기,2019,6,4,2600000,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79149,제주,충북,2019,3,3,557200,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,3
79150,제주,충북,2019,4,3,430200,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,3
79151,제주,충북,2019,5,6,922280,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,6
79152,제주,충북,2019,11,6,829520,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,6


In [ ]:
df_distance.loc[df_distance.SEX_CTGO_CD == 1, 'SEX_CTGO_CD'] = 0
df_distance.loc[df_distance.SEX_CTGO_CD == 2, 'SEX_CTGO_CD'] = 1


In [ ]:
df.columns

Index(['CARD_SIDO_NM', 'HOM_SIDO_NM', 'year', 'month', 'CSTMR_CNT', 'AMT',
       'CNT', '업종_건강보조식품 소매업', '업종_골프장 운영업', '업종_과실 및 채소 소매업',
       '업종_관광 민예품 및 선물용품 소매업', '업종_그외 기타 분류안된 오락관련 서비스업', '업종_그외 기타 스포츠시설 운영업',
       '업종_그외 기타 종합 소매업', '업종_기타 대형 종합 소매업', '업종_기타 수상오락 서비스업',
       '업종_기타 외국식 음식점업', '업종_기타 주점업', '업종_기타음식료품위주종합소매업', '업종_내항 여객 운송업',
       '업종_마사지업', '업종_면세점', '업종_비알콜 음료점업', '업종_빵 및 과자류 소매업', '업종_서양식 음식점업',
       '업종_수산물 소매업', '업종_슈퍼마켓', '업종_스포츠 및 레크레이션 용품 임대업', '업종_여관업', '업종_여행사업',
       '업종_욕탕업', '업종_육류 소매업', '업종_일반유흥 주점업', '업종_일식 음식점업', '업종_자동차 임대업',
       '업종_전시 및 행사 대행업', '업종_정기 항공 운송업', '업종_중식 음식점업', '업종_차량용 가스 충전업',
       '업종_차량용 주유소 운영업', '업종_체인화 편의점', '업종_택시 운송업', '업종_피자 햄버거 샌드위치 및 유사 음식점업',
       '업종_한식 음식점업', '업종_호텔업', '업종_화장품 및 방향제 소매업', '업종_휴양콘도 운영업', '나이_10s',
       '나이_20s', '나이_30s', '나이_40s', '나이_50s', '나이_60s', '나이_70s', '성별_1',
       '성별_2', '가구_1', '가구_2', '가구_3', '가구_4', '가구_5', 'SIDOCHAI', 'PERSON'],
      dtype='object')

In [ ]:
df_distance.PERSON = pd.qcut(df_distance.PERSON, 4, labels = False)

In [ ]:
df_distance.PERSON.unique()

array([0, 1, 2, 3])

In [ ]:
fn =['업종_건강보조식품 소매업', '업종_골프장 운영업', '업종_과실 및 채소 소매업',
       '업종_관광 민예품 및 선물용품 소매업', '업종_그외 기타 분류안된 오락관련 서비스업', '업종_그외 기타 스포츠시설 운영업',
       '업종_그외 기타 종합 소매업', '업종_기타 대형 종합 소매업', '업종_기타 수상오락 서비스업',
       '업종_기타 외국식 음식점업', '업종_기타 주점업', '업종_기타음식료품위주종합소매업', '업종_내항 여객 운송업',
       '업종_마사지업', '업종_면세점', '업종_비알콜 음료점업', '업종_빵 및 과자류 소매업', '업종_서양식 음식점업',
       '업종_수산물 소매업', '업종_슈퍼마켓', '업종_스포츠 및 레크레이션 용품 임대업', '업종_여관업', '업종_여행사업',
       '업종_욕탕업', '업종_육류 소매업', '업종_일반유흥 주점업', '업종_일식 음식점업', '업종_자동차 임대업',
       '업종_전시 및 행사 대행업', '업종_정기 항공 운송업', '업종_중식 음식점업', '업종_차량용 가스 충전업',
       '업종_차량용 주유소 운영업', '업종_체인화 편의점', '업종_택시 운송업', '업종_피자 햄버거 샌드위치 및 유사 음식점업',
       '업종_한식 음식점업', '업종_호텔업', '업종_화장품 및 방향제 소매업', '업종_휴양콘도 운영업', '나이_10s',
       '나이_20s', '나이_30s', '나이_40s', '나이_50s', '나이_60s', '나이_70s', '성별_1',
       '성별_2', '가구_1', '가구_2', '가구_3', '가구_4', '가구_5', 'SIDOCHAI', 'PERSON']

In [ ]:
train = df[df.year == 2019]
test = df[(df.year == 2020) & (df.month == 3)]

In [ ]:
xtrain = train[fn]
ytrain = np.array(train['AMT'])
xtest = test[fn]
ylabel = np.array(test['AMT'])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.metrics import make_scorer
from sklearn.linear_model import LinearRegression

In [ ]:
ytrain = np.log1p(ytrain)

In [ ]:
model = LinearRegression()

model.fit(xtrain, ytrain)
y_pred = model.predict(xtest)

# print('정확도 %.f' %accuracy_score(ylabel, y_pred))

In [ ]:
def rmsle(predicted_values, actual_values):
	# 넘파이로 배열 형태로 바꿔줌.  
  predicted_values = np.array(predicted_values)
  actual_values = np.array(actual_values)
    
  # 예측값과 실제 값에 1을 더하고 로그를 씌어줌 
  log_predict = np.log(predicted_values + 1)
  log_actual = np.log(actual_values + 1)
    
  # 위에서 계산한 예측값에서 실측값을 빼주고 제곱해줌
  difference = log_predict - log_actual
  difference = np.square(difference)
    
  # 평균을 냄
  mean_difference = difference.mean()
    
  # 다시 루트를 씌움
  score = np.sqrt(mean_difference)  
    
  return score

In [ ]:
rmsle(y_pred, ylabel)


11.372351583549442